<img src="https://www.iscte-iul.pt/assets/images/logo_iscte_detailed.svg" style="width: 450px;margin-top:30px;" align ="center">

<div style= "font-size: 35px;  margin-top:40px; font-weight:bold; font-family: 'Avenir Next LT Pro', sans-serif;"><center>Data Preparation & Feature Engineering: <strong>E-Commerce</strong></center></div>
<div style= "font-size: 30px; font-weight:bold; font-family: 'Avenir Next LT Pro', sans-serif;"><center>Organizing all variables and data to the models phase</center></div>

<div style= "font-size: 27px;font-weight:bold;line-height: 1.1; margin-top:40px; font-family: 'Avenir Next LT Pro', sans-serif;"><center>Processamento e Modelação de Big Data 2024/2025</center></div> <br>

   <div style= "font-size: 20px;font-weight:bold; font-family: 'Avenir Next LT Pro', sans-serif;"><center> Grupo 7:</center></div>
   <div><center> Diogo Freitas | 104841 </center></div>
   <div><center> João Francisco Botas | 104782 </center></div>
   <div><center> Miguel Gonçalves | 105944 </center></div>
   <div><center> Ricardo Galvão | 105285 </center></div>

### imports

In [1]:
# Importações Básicas
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import (
    StructType, StructField,
    StringType, LongType, DoubleType, TimestampType
)
import pandas as pd
import numpy as np
import os
import sys
import math
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
import matplotlib.pyplot as plt
import pyarrow

## 1. Recommendation Engine with PySpark

In [2]:
# Inicializa a sessão Spark
spark = SparkSession.builder.appName("PMBD - Data Preparation and Data Engineering") \
    .config("spark.sql.shuffle.partitions", "400") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "8") \
    .config("spark.executor.memoryOverhead", "4g") \
    .config("spark.executor.instances", "10") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")  # Pode ser ajustado para "INFO" durante o desenvolvimento

In [3]:
print(spark.sparkContext._jsc.sc().isStopped())  # False -> tudo bem; True -> Spark está desligado

False


In [4]:
data_dir = "../data/"

dataProcessed_dir = data_dir + "processed/"
dataRaw_dir = data_dir + "raw/"
# Criar diretórios se não existirem
os.makedirs(dataProcessed_dir, exist_ok=True)
os.makedirs(dataRaw_dir, exist_ok=True)

allData = False

schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", LongType(), True),
    StructField("category_id", LongType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("user_id", LongType(), True),
    StructField("user_session", StringType(), True)
])

if allData:
    # Lê o arquivo parquet completo - não recomendado para grandes volumes de dados
    df = spark.read.parquet(dataRaw_dir + "ec_total.parquet", schema=schema)
else:
    # Lê o arquivo parquet em partes - recomendado para grandes volumes de dados
    df_nov = spark.read.csv(dataRaw_dir + '2019-Nov.csv', schema=schema, header=True).limit(1000000)
    # df_oct = spark.read.csv(dataRaw_dir + '2019-Oct.csv', schema=schema, header=True)

In [5]:
df_nov.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_session: string (nullable = true)



In [6]:
df_nov.show(5, truncate=False)

+-------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code            |brand |price |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|2019-11-01 00:00:00|view      |1003461   |2053013555631882655|electronics.smartphone   |xiaomi|489.07|520088904|4d3b30da-a5e4-49df-b1a8-ba5943f1dd33|
|2019-11-01 00:00:00|view      |5000088   |2053013566100866035|appliances.sewing_machine|janome|293.65|530496790|8e5f4f83-366c-4f70-860e-ca7417414283|
|2019-11-01 00:00:01|view      |17302664  |2053013553853497655|NULL                     |creed |28.31 |561587266|755422e7-9040-477b-9bd2-6a6e8fd97387|
|2019-11-01 00:00:01|view      |3601530   |2053013563810775923|appliances.kitchen.washer|lg   

In [7]:
ec = df_nov
ec.show()

+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|  17302664|2053013553853497655|                NULL|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.s

## 2. Eliminação de observações com NaN na coluna category_code

In [8]:
ec.filter(F.isnull(col("category_code"))).head(5)

[Row(event_time=datetime.datetime(2019, 11, 1, 0, 0, 1), event_type='view', product_id=17302664, category_id=2053013553853497655, category_code=None, brand='creed', price=28.31, user_id=561587266, user_session='755422e7-9040-477b-9bd2-6a6e8fd97387'),
 Row(event_time=datetime.datetime(2019, 11, 1, 0, 0, 2), event_type='view', product_id=15900065, category_id=2053013558190408249, category_code=None, brand='rondell', price=30.86, user_id=518574284, user_session='5e6ef132-4d7c-4730-8c7f-85aa4082588f'),
 Row(event_time=datetime.datetime(2019, 11, 1, 0, 0, 2), event_type='view', product_id=12708937, category_id=2053013553559896355, category_code=None, brand='michelin', price=72.72, user_id=532364121, user_session='0a899268-31eb-46de-898d-09b2da950b24'),
 Row(event_time=datetime.datetime(2019, 11, 1, 0, 0, 4), event_type='view', product_id=34600011, category_id=2060981320581906480, category_code=None, brand=None, price=20.54, user_id=512416379, user_session='4dfe2c67-e537-4dc2-ae69-0b65263db0

In [9]:
# Remove observações com categoria nula
ec_clean = ec.filter(~isnull(col("category_code"))).filter(~isnan(col("category_code")))

In [10]:
ec.count(), ec_clean.count()

(1000000, 683023)

In [11]:
ec_clean.show()

+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...| xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...| janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|     lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.smart...| xiaomi|183.27|558856683|313628f1-68b8-460...|
|2019-11-01 00:00:01|      view|   1306894|2053013558920217191|  computers.notebook

## 3. Separação da categoria do produto em sub-categorias

In [12]:
# Separação do category_code através do '.'
split_col = split(ec_clean['category_code'], '\.')

# Separação do category_code em categoria e sub categorias
ec_clean = ec_clean.withColumn('main_category', split_col.getItem(0)) \
       .withColumn('sub_category_1', split_col.getItem(1)) \
       .withColumn('sub_category_2', split_col.getItem(2))

# Remoção da category_code original 
ec_clean = ec_clean.drop('category_code')

In [13]:
ec_clean.show()

+-------------------+----------+----------+-------------------+-------+------+---------+--------------------+-------------+--------------+--------------+
|         event_time|event_type|product_id|        category_id|  brand| price|  user_id|        user_session|main_category|sub_category_1|sub_category_2|
+-------------------+----------+----------+-------------------+-------+------+---------+--------------------+-------------+--------------+--------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655| xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|  electronics|    smartphone|          NULL|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035| janome|293.65|530496790|8e5f4f83-366c-4f7...|   appliances|sewing_machine|          NULL|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|     lg|712.87|518085591|3bfb58cd-7892-48c...|   appliances|       kitchen|        washer|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655| xiaomi|183.2

In [14]:
# Number of observations with sub categories
print(ec_clean.filter(~isnull(col("sub_category_1"))).count())
print(ec_clean.filter(~isnull(col("sub_category_2"))).count())

683023
271616


In [15]:
ec_clean.filter(col("main_category")=="medicine").show()

+-------------------+----------+----------+-------------------+--------+------+---------+--------------------+-------------+--------------+--------------+
|         event_time|event_type|product_id|        category_id|   brand| price|  user_id|        user_session|main_category|sub_category_1|sub_category_2|
+-------------------+----------+----------+-------------------+--------+------+---------+--------------------+-------------+--------------+--------------+
|2019-11-01 00:44:39|      view|  25800007|2053013562292437791|   omron| 47.08|518484905|806e4279-33f2-43a...|     medicine|         tools|     tonometer|
|2019-11-01 01:57:09|      view|  25800033|2053013562292437791|csmedica| 30.69|543220642|16894153-47f1-49a...|     medicine|         tools|     tonometer|
|2019-11-01 01:57:36|      view|  25800033|2053013562292437791|csmedica| 30.69|543220642|16894153-47f1-49a...|     medicine|         tools|     tonometer|
|2019-11-01 02:06:19|      view|  25800003|2053013562292437791|   omro

In [16]:
top_10_cat = ec_clean.filter(col("event_type") == "view") \
  .repartition(24, 'main_category')\
  .groupBy("main_category") \
  .count() \
  .orderBy("count", ascending=False) \
  .select("main_category") \
  .collect()

top_10_cat = [x[0] for x in top_10_cat][:10]
top_10_cat

['electronics',
 'appliances',
 'computers',
 'furniture',
 'apparel',
 'auto',
 'construction',
 'kids',
 'accessories',
 'sport']

## 4. Criação do dataframe para clientes

In [17]:
user_df = ec_clean.filter((col("event_type") == "view") & (col("main_category").isin(top_10_cat))) \
    .repartition(24, "user_id") \
    .groupBy("user_id") \
    .agg(
        avg("price").alias("average_price"),
        count("*").alias("views")
    ) \
    .withColumn("average_price", round(col("average_price"), 3))

user_df.show()

+---------+-------------+-----+
|  user_id|average_price|views|
+---------+-------------+-----+
|565404816|     1132.465|    2|
|519277091|      1091.33|    1|
|539701280|      342.709|   23|
|566280946|       180.18|    1|
|565921426|       165.26|    1|
|566280676|        30.89|    1|
|541781462|      203.877|   15|
|566281299|       165.26|    1|
|562958505|      171.543|    3|
|512458378|      731.833|    8|
|566281250|       287.83|    2|
|512461324|       128.42|    1|
|517119857|      334.783|    4|
|516161337|      266.113|    9|
|527396367|        42.99|    1|
|566282003|      1348.61|    1|
|543638253|      1029.37|    1|
|512712071|      301.934|   26|
|544248389|      195.246|    7|
|513590282|       59.133|    8|
+---------+-------------+-----+
only showing top 20 rows



In [18]:
avg_price_df = ec_clean.filter((col("event_type") == "view")) \
 .groupBy("user_id") \
 .agg(round(avg("price"), 3).alias("average_price"),
      count("*").alias("total_views"))

pivot_views_df = ec_clean.filter(
    (col("event_type") == "view") & 
    (col("main_category").isin(top_10_cat))
).groupBy("user_id") \
 .pivot("main_category", top_10_cat) \
 .agg(count("*"))

user_df = avg_price_df.join(pivot_views_df, on="user_id", how="left")
user_df = user_df.fillna(0)
user_df.show()

+---------+-------------+-----------+-----------+----------+---------+---------+-------+----+------------+----+-----------+-----+
|  user_id|average_price|total_views|electronics|appliances|computers|furniture|apparel|auto|construction|kids|accessories|sport|
+---------+-------------+-----------+-----------+----------+---------+---------+-------+----+------------+----+-----------+-----+
|513200477|       29.611|          9|          0|         9|        0|        0|      0|   0|           0|   0|          0|    0|
|513741043|       514.79|          1|          0|         0|        1|        0|      0|   0|           0|   0|          0|    0|
|514028527|      461.737|          7|          0|         0|        7|        0|      0|   0|           0|   0|          0|    0|
|518085591|      399.635|         10|          0|         1|        0|        9|      0|   0|           0|   0|          0|    0|
|518427361|       155.11|          1|          0|         1|        0|        0|      0|  

In [19]:
user_df.count()

128528

In [20]:
# write in a parquet file
user_df.write.parquet(data_dir + "user_df.parquet", mode="overwrite")

## 5. Tratamento dos dados para os algoritmos de recomendação

In [21]:
# Plot events distribution

events = df_nov.groupBy('event_type').count().toPandas()
events

,event_type,count
0,view,967225
1,cart,14958
2,purchase,17817


### 5.1. Preprocess

In [22]:
def preprocess(df):
    
    # Change data types
    df = df.withColumn('event_time', to_timestamp('event_time'))
    df = df.withColumn('user_id', col('user_id').cast('integer'))
    df = df.withColumn('product_id', col('product_id').cast('integer'))
    df = df.withColumn('category_id', col('category_id').cast('long'))
    
    # Limit the number of carts to 1 per session for each user-product pair
    cart_df = df.filter(col('event_type') == 'cart')
    df = df.filter(col('event_type') != 'cart')
    cart_df = cart_df.dropDuplicates(subset=['product_id', 'user_id', 'user_session'])
    df = df.union(cart_df)
    
    # Split category codes into sub categories
    #df = df.withColumn('category', split(df['category_code'], '\.').getItem(0)) \
    #   .withColumn('sub_category', split(df['category_code'], '\.').getItem(1)) \
    #   .withColumn('sub_sub_category', split(df['category_code'], '\.').getItem(2))

    return df

In [23]:
df = preprocess(df_nov)
df.show(5, truncate=False)

+-------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code            |brand |price |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-------------------------+------+------+---------+------------------------------------+
|2019-11-01 00:00:00|view      |1003461   |2053013555631882655|electronics.smartphone   |xiaomi|489.07|520088904|4d3b30da-a5e4-49df-b1a8-ba5943f1dd33|
|2019-11-01 00:00:00|view      |5000088   |2053013566100866035|appliances.sewing_machine|janome|293.65|530496790|8e5f4f83-366c-4f70-860e-ca7417414283|
|2019-11-01 00:00:01|view      |17302664  |2053013553853497655|NULL                     |creed |28.31 |561587266|755422e7-9040-477b-9bd2-6a6e8fd97387|
|2019-11-01 00:00:01|view      |3601530   |2053013563810775923|appliances.kitchen.washer|lg   

### 5.2. Feature Engineering

Podemos extrair várias características para utilizadores, produtos e categorias utilizando os dados comportamentais:

- **Visualizações/carrinhos/compras**: O número de vezes que um utilizador interagiu com um produto.

- **Produtos distintos visualizados/adicionados ao carrinho/comprados**: O número de produtos únicos que um utilizador visualizou, adicionou ao carrinho ou comprou.

- **Preço médio (relativo) visualizado/comprado**: O preço médio dos produtos que um utilizador visualizou ou comprou.

- **Taxas de interação**: São características derivadas que fornecem informações sobre o comportamento dos utilizadores, como a taxa de compras por visualização, carrinho por visualização e compras por carrinho.

In [24]:
def product_features(df):

    # Calculate several metrics for products with the aggregate function
    df = df.groupby('product_id').agg(first('category_id').alias('category_id'),
                                      first('category_code').alias('category_code'),
                                      count(when(df['event_type'] == 'view', True)).alias('views'),
                                      count(when(df['event_type'] == 'cart', True)).alias('carts'),
                                      count(when(df['event_type'] == 'purchase', True)).alias('purchases'),
                                      mean('price').alias('price'),
                                      min('event_time').alias('first_date'),
                                      max('event_time').alias('last_date'))
    
    # Calculate interaction rates
    df = df.withColumn('purchase_per_view', df['purchases'] / df['views'])
    df = df.withColumn('cart_per_view', df['carts'] / df['views'])
    df = df.withColumn('purchase_per_cart', when(df['carts'] == 0, df['purchases']).otherwise(df['purchases'] / df['carts']))
    
    return df

In [25]:
def category_features(df):
    
    # Calculate the average product price for each category
    products = df.dropDuplicates(subset=['product_id'])
    products = products.groupby('category_id').agg(avg('price').alias('average_price'))

    # Calculate several metrics for categories with the aggregate function
    df = df.groupby('category_id').agg(first('category_code').alias('category_code'),
                                       countDistinct('product_id').alias('number_of_products'),
                                       count(when(df['event_type'] == 'view', True)).alias('views'),
                                       count(when(df['event_type'] == 'cart', True)).alias('carts'),
                                       count(when(df['event_type'] == 'purchase', True)).alias('purchases'))
    
    # Calculate interaction rates
    df = df.withColumn('purchase_per_view', df['purchases'] / df['views'])
    df = df.withColumn('cart_per_view', df['carts'] / df['views'])
    df = df.withColumn('purchase_per_cart', when(df['carts'] == 0, df['purchases']).otherwise(df['purchases'] / df['carts']))
    
    df = df.join(products, on='category_id')
    
    return df

In [26]:
def user_features(df):

    # Calculate several metrics for users with the aggregate function
    df = df.groupby('user_id').agg(count(when(df['event_type'] == 'view', True)).alias('views'),
                                   count(when(df['event_type'] == 'cart', True)).alias('carts'),
                                   count(when(df['event_type'] == 'purchase', True)).alias('purchases'),
                                   countDistinct(when(df['event_type'] == 'view', col('product_id'))).alias('distinct_products_viewed'),
                                   countDistinct(when(df['event_type'] == 'cart', col('product_id'))).alias('distinct_products_carted'),
                                   countDistinct(when(df['event_type'] == 'purchase', col('product_id'))).alias('distinct_products_purchased'),
                                   mean(when(df['event_type'] == 'view', col('price'))).alias('average_price_viewed'),
                                   mean(when(df['event_type'] == 'purchase', col('price'))).alias('average_price_purchased'),
                                   mean(when(df['event_type'] == 'view', col('relative_price'))).alias('avg_relative_price_viewed'),
                                   mean(when(df['event_type'] == 'purchase', col('relative_price'))).alias('avg_relative_price_purchased'),
                                   min('event_time').alias('first_date'),
                                   max('event_time').alias('last_date'))
    
    # Calculate interaction rates
    df = df.withColumn('purchase_per_view', when(df['views'] == 0, df['purchases']).otherwise(df['purchases'] / df['views']))
    df = df.withColumn('cart_per_view', when(df['views'] == 0, df['carts']).otherwise(df['carts'] / df['views']))
    df = df.withColumn('purchase_per_cart', when(df['carts'] == 0, df['purchases']).otherwise(df['purchases'] / df['carts']))
    
    return df

Alguns produtos e categorias têm muito poucas interações, o que torna as suas taxas de interação pouco fiáveis. Por exemplo, um produto que foi visualizado apenas 2 vezes e comprado 1 vez teria uma taxa de compras por visualização de 0.5, o que é muito elevado. Por isso, é importante aplicar *smoothing* para evitar *overfitting* em amostras pequenas.

O *smoothing* funciona ao calcular uma média ponderada entre a taxa do produto e a taxa da categoria. O peso dado à taxa do produto depende do número de interações — quanto mais interações, maior o peso. O mesmo tipo de *smoothing* é aplicado às categorias, usando as taxas médias globais de interação.

In [27]:
# Calculate smoothed rates for categories with small sample size using means of all products to combat overfitting
    
def category_smoothener(categories, mean, attr, rate, min_sample_size=1000):
    
    # Weighted average of category value and overall mean
    categories = categories.withColumn(rate, when(categories[attr] < min_sample_size, ((categories[rate] * categories[attr]) + (mean * (min_sample_size - categories[attr]))) / min_sample_size).otherwise(categories[rate]))
    
    return categories

In [28]:
# Calculate smoothed rates for products with small sample size using category means to combat overfitting
    
def product_smoothener(products, categories, attr, rate, min_sample_size=1000):
    
    category_rate = rate + '_cat'
    categories = categories.withColumnRenamed(rate, category_rate)
    products = products.join(categories['category_id', category_rate], on='category_id')
    
    # Weighted average of product value and category value
    products = products.withColumn(rate, when(products[attr] < min_sample_size, ((products[rate] * products[attr]) + (products[category_rate] * (min_sample_size - products[attr]))) / min_sample_size).otherwise(products[rate]))
    
    products = products.drop(category_rate)
    return products

Devemos calcular o preço relativo de cada produto em relação à sua categoria, escalando o seu preço com base na mediana e no intervalo interquartil (IQR) dos preços dessa categoria. A mediana é usada como medida de tendência central e o IQR como medida de variabilidade.

Este escalamento permite compensar diferenças nos intervalos de preços entre categorias, possibilitando uma comparação mais precisa e imparcial dos preços dos produtos. Os preços relativos serão usados para determinar se os utilizadores tendem a interagir com produtos mais baratos ou mais caros.

In [29]:
# Calculate the price of each product relative to its category using the formula: (price - median) / IQR

def calculate_relative_price(products):
    
    categories = products.groupby('category_id').agg(percentile_approx('price', 0.25, 1000).alias('Q1'),
                                                     percentile_approx('price', 0.5, 1000).alias('median'),
                                                     percentile_approx('price', 0.75, 1000).alias('Q3'))
    # Interquartile range
    categories = categories.withColumn('IQR', col('Q3') - col('Q1'))
    categories = categories.withColumn('IQR', when(col('IQR') < 1, 1).otherwise(col('IQR')))
    
    # Calculate relative price
    products = products.join(categories, on='category_id')
    products = products.withColumn('relative_price', (col('price') - col('median')) / col('IQR'))
    
    # In order to avoid extreme values, set the max possible value to 5
    products = products.withColumn('relative_price', when(col('relative_price') > 5, 5).otherwise(col('relative_price')))

    # Set the min possible value to -5
    products = products.withColumn('relative_price', when(col('relative_price') < -5, -5).otherwise(col('relative_price')))

    products = products.select('product_id', 'relative_price')
    return products

In [30]:
# Extract product and category features
products = product_features(df)
products.show(5, truncate=False)

+----------+-------------------+----------------------+-----+-----+---------+------------------+-------------------+-------------------+--------------------+-------------+-----------------+
|product_id|category_id        |category_code         |views|carts|purchases|price             |first_date         |last_date          |purchase_per_view   |cart_per_view|purchase_per_cart|
+----------+-------------------+----------------------+-----+-----+---------+------------------+-------------------+-------------------+--------------------+-------------+-----------------+
|1000978   |2053013555631882655|electronics.smartphone|11   |0    |0        |301.13999999999993|2019-11-01 03:42:18|2019-11-01 14:07:25|0.0                 |0.0          |0.0              |
|1002100   |2053013555631882655|electronics.smartphone|147  |0    |1        |370.6399999999996 |2019-11-01 02:07:31|2019-11-01 15:31:42|0.006802721088435374|0.0          |1.0              |
|1002102   |2053013555631882655|electronics.smartp

In [31]:
categories = category_features(df)
categories.show(5, truncate=False)

+-------------------+-----------------------+------------------+-----+-----+---------+--------------------+--------------------+-----------------+------------------+
|category_id        |category_code          |number_of_products|views|carts|purchases|purchase_per_view   |cart_per_view       |purchase_per_cart|average_price     |
+-------------------+-----------------------+------------------+-----+-----+---------+--------------------+--------------------+-----------------+------------------+
|2060237588744111062|NULL                   |2                 |33   |0    |0        |0.0                 |0.0                 |0.0              |734.4499999999999 |
|2098563460336976001|construction.tools.pump|8                 |15   |0    |0        |0.0                 |0.0                 |0.0              |10.085            |
|2053013564968403895|NULL                   |128               |593  |0    |5        |0.008431703204047217|0.0                 |5.0              |98.701171875      |
|213

In [32]:
# Calculate relative prices and append it to the main dataframe and product features
relative_prices = calculate_relative_price(products)
relative_prices.show(5, truncate=False)

+----------+-------------------+
|product_id|relative_price     |
+----------+-------------------+
|8600043   |-0.5677202895991524|
|8600234   |3.3114956736712    |
|8600094   |0.40950026487727353|
|8600147   |0.0                |
|8600116   |-0.5449408440755783|
+----------+-------------------+
only showing top 5 rows



In [33]:
df = df.join(relative_prices, on='product_id')
df.show(5, truncate=False)

+----------+-------------------+----------+-------------------+-----------------------------+-------+------+---------+------------------------------------+--------------------+
|product_id|event_time         |event_type|category_id        |category_code                |brand  |price |user_id  |user_session                        |relative_price      |
+----------+-------------------+----------+-------------------+-----------------------------+-------+------+---------+------------------------------------+--------------------+
|1004739   |2019-11-01 00:04:51|view      |2053013555631882655|electronics.smartphone       |xiaomi |203.09|534313202|86171b68-6479-4b53-b129-5be669c12493|-0.06399516343719788|
|1004739   |2019-11-01 00:09:56|view      |2053013555631882655|electronics.smartphone       |xiaomi |203.09|561121118|9df1f0b8-7501-432b-b622-cc5fc3398faa|-0.06399516343719788|
|3701000   |2019-11-01 00:11:06|view      |2053013565983425517|appliances.environment.vacuum|samsung|669.18|5126896

In [34]:
products = products.join(relative_prices, on='product_id')
products.show(5, truncate=False)

+----------+-------------------+--------------------------+-----+-----+---------+------------------+-------------------+-------------------+-----------------+-------------+-----------------+-------------------+
|product_id|category_id        |category_code             |views|carts|purchases|price             |first_date         |last_date          |purchase_per_view|cart_per_view|purchase_per_cart|relative_price     |
+----------+-------------------+--------------------------+-----+-----+---------+------------------+-------------------+-------------------+-----------------+-------------+-----------------+-------------------+
|8600043   |2053013552226107603|appliances.environment.fan|5    |0    |0        |14.13             |2019-11-01 08:13:34|2019-11-01 14:42:59|0.0              |0.0          |0.0              |-0.5677202895991524|
|8600234   |2053013552226107603|appliances.environment.fan|3    |0    |0        |233.81000000000003|2019-11-01 08:12:36|2019-11-01 15:22:13|0.0             

In [35]:
# Calculate the average interaction rates for all products to smooth category features with low interaction

avg_purchase_per_view = events[events['event_type'] == 'purchase']['count'].values[0] / events[events['event_type'] == 'view']['count'].values[0]
avg_cart_per_view = events[events['event_type'] == 'cart']['count'].values[0] / events[events['event_type'] == 'view']['count'].values[0]
avg_purchase_per_cart = events[events['event_type'] == 'purchase']['count'].values[0] / events[events['event_type'] == 'cart']['count'].values[0]

categories = category_smoothener(categories, avg_purchase_per_view, 'views', 'purchase_per_view', 2000)
categories = category_smoothener(categories, avg_cart_per_view, 'views', 'cart_per_view', 2000)
categories = category_smoothener(categories, avg_purchase_per_cart, 'carts', 'purchase_per_cart', 200)

In [36]:
# Smooth product features with low interaction using corresponding category features

products = product_smoothener(products, categories, 'views', 'purchase_per_view', 1000)
products = product_smoothener(products, categories, 'views', 'cart_per_view', 1000)
products = product_smoothener(products, categories, 'carts', 'purchase_per_cart', 100)

In [37]:
# Extract user features

users = user_features(df)
users.show(5, truncate=False)

+---------+-----+-----+---------+------------------------+------------------------+---------------------------+--------------------+-----------------------+-------------------------+----------------------------+-------------------+-------------------+-----------------+-------------+-----------------+
|user_id  |views|carts|purchases|distinct_products_viewed|distinct_products_carted|distinct_products_purchased|average_price_viewed|average_price_purchased|avg_relative_price_viewed|avg_relative_price_purchased|first_date         |last_date          |purchase_per_view|cart_per_view|purchase_per_cart|
+---------+-----+-----+---------+------------------------+------------------------+---------------------------+--------------------+-----------------------+-------------------------+----------------------------+-------------------+-------------------+-----------------+-------------+-----------------+
|563247292|2    |0    |0        |2                       |0                       |0          

In [38]:
# Top 5 products in terms of purchases

products.sort('purchases', ascending=False).toPandas().head(5)

,category_id,product_id,category_code,views,carts,purchases,price,first_date,last_date,purchase_per_view,cart_per_view,purchase_per_cart,relative_price
0,2053013555631882655,1004856,electronics.smartphone,10307,519,780,128.318487,2019-11-01 00:00:23,2019-11-01 15:32:04,0.075677,0.050354,1.502890,-0.242941
1,2053013555631882655,1004767,electronics.smartphone,9033,442,503,242.379504,2019-11-01 00:00:30,2019-11-01 15:32:07,0.055685,0.048932,1.138009,0.035576
2,2053013555631882655,1005115,electronics.smartphone,9181,389,430,946.502500,2019-11-01 00:00:10,2019-11-01 15:32:06,0.046836,0.042370,1.105398,1.754920
3,2053013555631882655,1004833,electronics.smartphone,5009,311,375,171.951703,2019-11-01 00:00:49,2019-11-01 15:32:05,0.074865,0.062088,1.205788,-0.136396
4,2053013555631882655,1002544,electronics.smartphone,4568,235,299,458.266535,2019-11-01 00:00:14,2019-11-01 15:31:53,0.065455,0.051445,1.272340,0.562734


In [39]:
# write in a parquet file
products.write.parquet(dataProcessed_dir + "products.parquet", mode="overwrite")
users.write.parquet(dataProcessed_dir + "users.parquet", mode="overwrite")
categories.write.parquet(dataProcessed_dir + "categories.parquet", mode="overwrite")
df.write.parquet(dataProcessed_dir + "ec.parquet", mode="overwrite")

In [40]:
spark.catalog.clearCache()